In [1]:
import datetime
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import phik
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'
sns.set_theme(context='talk', style='whitegrid', palette='deep')
plt.rcParams['figure.figsize'] = 10, 7
plt.rcParams['font.size'] = 20
plt.rcParams['axes.labelsize'] = 25
plt.rcParams['figure.titlesize'] = 32
plt.rcParams['axes.titlesize'] = 32
plt.rcParams['savefig.format'] = 'pdf'
plt.rcParams['figure.autolayout'] = 'true'
plt.rcParams['figure.frameon'] = 'false'
plt.rcParams['axes.spines.left'] = 'false'
plt.rcParams['axes.spines.right'] = 'false'
plt.rcParams['axes.spines.top'] = 'false'
plt.rcParams['legend.fancybox'] = 'false'
plt.rcParams['axes.spines.bottom'] = 'false'
plt.rcParams['font.size'] = 20
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
# для графиков, где надо много цветов, юзайте воть:
sns.set_palette(sns.color_palette('deep'))
# а по дефолту воть:
sns.set_palette(sns.color_palette('BuGn_r', n_colors=10)[2::3])
pd.set_option('display.max_columns', 60)

In [2]:
feeding_details_22 = pd.read_csv('datasets/2022-feeding-tasks-details.csv', on_bad_lines='skip')
feeding_22 = pd.read_csv('datasets/2022-feeding-tasks.csv', on_bad_lines='skip')

feeding_details_23 = pd.read_csv('datasets/2023-feeding-tasks-details.csv', on_bad_lines='skip')
feeding_23 = pd.read_csv('datasets/2023-feeding-tasks.csv', on_bad_lines='skip')

feeding_details_24 = pd.read_csv('datasets/2024-feeding-tasks-details.csv', on_bad_lines='skip')
feeding_24 = pd.read_csv('datasets/2024-feeding-tasks.csv', on_bad_lines='skip')

feeding_details_25 = pd.read_csv('datasets/2025-feeding-tasks-details.csv', on_bad_lines='skip')
feeding_25 = pd.read_csv('datasets/2025-feeding-tasks.csv', on_bad_lines='skip')

In [3]:
monthly_feeding = pd.read_excel('datasets/Ekoniva_dataset.xlsx', sheet_name='Feeding')
herd_metrics = pd.read_excel('datasets/Ekoniva_dataset.xlsx', sheet_name='Herd maintenance').replace('-', np.nan)
production_indicators = pd.read_excel('datasets/Ekoniva_dataset.xlsx', sheet_name='Dairy indicators').replace('-', np.nan)

In [4]:
feeding_details_23.IngredientName.unique()

array(['Солома (общ.)', 'Кукуруза', 'Комбикорм 10 группы', 'Шрот соевый',
       'Шрот рапсовый', 'Кукуруза плющеная', '2201.02.01.01.2.22',
       '2201.03.05.02.1.22', 'Патока свекловичная', 'Вода',
       'Комбикорм 2 группы', 'Сено луговое', 'Комбикорм 3.1 группы',
       'Премикс Молодняк 9-12', 'Остатки', 'Молоко', 'ЗЦМ',
       'Шаумацид Дринк', 'Электролит', 'Стартер', 'Бустер Милк',
       'Биотек Микс', '2221.01.01.01.1.22', '2221.03.05.02.1.22',
       '5902.01.01.01.1.22', 'Жом свекловичный сухой',
       '5313.06.05.02.1.22', '2221.04.05.02.1.21', '2501.03.01.01.1.22',
       'Комбикорм №11 ЭНА Восток', '2501.06.05.02.1.22',
       '3636.01.03.01.1.21', '2501.04.10.01.1.22',
       'Комбикорм № 3 престартер гранул.', '3645.01.01.01.1.22', 'Сорго',
       'Комбикорм 11 группы', '3645.03.05.02.1.22', 'Соль',
       'Премикс Дойный А', 'Сода', 'Пшеница', '5204.01.01.01.1.22',
       '5100.01.01.01.1.22', '5001.02.05.02.1.22', 'Мел',
       'Премикс Транзит А', 'Мегабуст Румен

In [5]:
def merge_tables() -> pd.DataFrame:
    """
    Merges all feeding dataframes with all details dataframes
    returns: merged dataframe
    rtype: pd.DataFrame
    """
    feeding_22["year"] = "2022"
    feeding_23["year"] = "2023"
    feeding_24["year"] = "2024"
    feeding_25["year"] = "2025"
    feeding_details_22["year"] = "2022"
    feeding_details_23["year"] = "2023"
    feeding_details_24["year"] = "2024"
    feeding_details_25["year"] = "2025"

    feeding_all = pd.concat([feeding_22, feeding_23, feeding_24, feeding_25], ignore_index=True)
    feeding_details_all = pd.concat([feeding_details_22, feeding_details_23, feeding_details_24, feeding_details_25], ignore_index=True)
    
    return feeding_all.merge(feeding_details_all, on=["FeedingTaskID", "SectionID", "year", "PhysiologicalGroupID", "PhysiologicalGroupName"], how="left"), feeding_all, feeding_details_all

In [6]:
monthly = pd.read_csv('monthly_vectors.csv')

In [7]:
feeding_and_details, feeding_all, feeding_details_all = merge_tables()

In [8]:
monthly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1733 entries, 0 to 1732
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                1733 non-null   int64  
 1   year_month                1733 non-null   object 
 2   FarmName                  1733 non-null   object 
 3   Concentrate               1733 non-null   float64
 4   Forage                    1733 non-null   float64
 5   Medication                1733 non-null   float64
 6   Premixture                1733 non-null   float64
 7   VitaminMineral            1733 non-null   float64
 8   Water                     1733 non-null   float64
 9   dist_to_center_euclid     1733 non-null   float64
 10  dist_to_center_manhattan  1733 non-null   float64
 11  pca1                      1733 non-null   float64
 12  pca2                      1733 non-null   float64
 13  pca_drift                 1680 non-null   float64
 14  cluster_

In [9]:
feeding_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155378 entries, 0 to 2155377
Data columns (total 14 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   FeedingTaskID                object 
 1   Date                         object 
 2   FarmName                     object 
 3   FeedNumber                   int64  
 4   SectionID                    int64  
 5   PhysiologicalGroupID         int64  
 6   PhysiologicalGroupName       object 
 7   PhysiologicalGroupHeadCount  int64  
 8   Appetite                     float64
 9   RationName                   object 
 10  RationPart                   float64
 11  TotalWeight_kg               float64
 12  CompletedAt                  object 
 13  year                         object 
dtypes: float64(3), int64(4), object(7)
memory usage: 230.2+ MB


In [10]:
feeding_details_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16443079 entries, 0 to 16443078
Data columns (total 9 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   FeedingTaskID           object 
 1   SectionID               int64  
 2   PhysiologicalGroupID    int64  
 3   PhysiologicalGroupName  object 
 4   IngredientID            int64  
 5   IngredientName          object 
 6   IngredientType          object 
 7   PhysicalWeight_kg       float64
 8   year                    object 
dtypes: float64(1), int64(3), object(5)
memory usage: 1.1+ GB


In [11]:
feeding_details_all.IngredientName.unique().shape

(1852,)

In [12]:
import re
from collections import Counter
def normalize_name(s):
    if pd.isna(s):
        return ""
    s = str(s).strip()
    s = s.replace("//", "/")
    s = re.sub(r"\s+", " ", s)
    s = s.lower()
    return s
feeding_details_all['norm_ingr_name'] = feeding_details_all['IngredientName'].apply(normalize_name)

In [13]:
code_re = re.compile(r'^\d+(?:\.\d+)+$') #Честно сам писал (100%)
feeding_details_all['is_code'] = feeding_details_all['norm_ingr_name'].str.match(code_re)
feeding_details_all['is_code'].sum()

np.int64(5111456)

In [14]:
feeding_details_all['norm_ingr_name']

0                  солома покупная
1               3645.01.01.01.1.20
2              комбикорм 10 группы
3                   кукуруза сухая
4                шрот подсолнечный
                     ...          
16443074             солома (общ.)
16443075        6203.01.01.01.1.25
16443076    жом свекловичный сухой
16443077     премикс молодняк 6-24
16443078        6202.03.05.02.1.24
Name: norm_ingr_name, Length: 16443079, dtype: object

In [15]:
smth = pd.ExcelFile("datasets/Ekoniva_dataset.xlsx")
nsi = smth.sheet_names[0]

In [16]:
cultures = pd.read_excel(
    smth,
    sheet_name=nsi,
    usecols="B:C",
    skiprows=81,
    nrows=40 
)
cultures.columns = ["code", "name"]
cultures.head()

,code,name
0,1.0,Люцерна
1,2.0,Эспарцет
2,3.0,Многолетние травы
3,4.0,Луговые травы
4,5.0,Кукуруза


In [17]:
razdels = pd.read_excel(
    smth,
    sheet_name=nsi,
    usecols="E:H",
    skiprows=81,
    nrows=88  
)
razdels.columns = ["code", "region", "prop_farm_name", "farm_name"]
razdels.head()


,code,region,prop_farm_name,farm_name
0,2501,Воронеж,ЭНА Восток,ЖК Бобров
1,2371,Воронеж,ЭНА Восток,ЖК Верхний Икорец
2,2281,Воронеж,ЭНА Восток,МТФ Владимировка
3,2351,Воронеж,ЭНА Восток,МТФ Петропавловка
4,3637,Воронеж,ЭНА Восток,ЖК Бобров 2


In [18]:
feed_type = pd.read_excel(
    smth,
    sheet_name=nsi,
    usecols="B:C",
    skiprows=123,
    nrows=10
)
feed_type.columns = ["code", "feed_name"]
feed_type.head()

,code,feed_name
0,1.0,Сенаж
1,2.0,Силос
2,3.0,Сено
3,4.0,Солома
4,5.0,Концентраы


In [19]:
cultures_map = cultures.set_index('code')['name'].to_dict()
feed_type_map = feed_type.set_index('code')['feed_name'].to_dict()

In [20]:
def decode_ingr_code(code):
    if pd.isna(code) or not isinstance(code, str):
        return code
    parts = code.split('.')
    if len(parts) < 4:
        return code
    try:
        culture_code = float(parts[2])
        feed_code = float(parts[3])
    except ValueError:
        return code    
    culture_name = cultures_map.get(culture_code, str(culture_code))
    feed_name = feed_type_map.get(feed_code, str(feed_code))
    return f"{culture_name} {feed_name}".lower()

In [21]:
feed_type_map

{1.0: 'Сенаж',
 2.0: 'Силос',
 3.0: 'Сено',
 4.0: 'Солома',
 5.0: 'Концентраы',
 6.0: 'Консервированное зерно',
 7.0: 'Корнаж',
 8.0: 'Сенаж в плёнке',
 9.0: 'Сенаж органический',
 nan: nan}

In [22]:
feeding_details_all['decoded_name'] = feeding_details_all.apply(
    lambda row: decode_ingr_code(row['norm_ingr_name']) if row['is_code'] else row['norm_ingr_name'],
    axis=1
)

feeding_details_all[['norm_ingr_name', 'decoded_name']].head(20)

,norm_ingr_name,decoded_name
0,солома покупная,солома покупная
1,3645.01.01.01.1.20,люцерна сенаж
2,комбикорм 10 группы,комбикорм 10 группы
3,кукуруза сухая,кукуруза сухая
4,шрот подсолнечный,шрот подсолнечный
5,шрот соевый,шрот соевый
6,жмых рапсовый,жмых рапсовый
7,жир защищённый,жир защищённый
8,2262.01.05.02.1.21,кукуруза силос
9,вода,вода


In [92]:
GROUP_KEYWORDS = {
    "forage": [
        "силос", "сенаж", "сено", "люцерн", "солома", "зеленая масса", "зел масса", "сило", "сорго",
        "сил", "сен", "люц", "трав", "зелёная масса", "клевер", "тимофеевк", "злаковая смесь",
        "солома пшеницы", "солома ячменя", "сенокос", "сенок", "тюк", "тюкован", "травосмесь",
        "сенокуб", "сеносилос", "рожь", "з-с рож 01", "курочкино", "к-ж", "soloma", "cилос", "01.01", "02.01", "03.05", "12.01"
    ],
    "energy": [
        "кукуруза", "пшениц", "ячмень", "овёс", "кукуруза плющ", "зерно", "зerno",
        "концентрат углевод", "глюгтен", "глютен", "корнаж", "карнаж", "патока свекловичная",
        "глицерин", "патока", "с-с кукурузный 1",
        "corn", "wheat", "barley", "oats", "grain", "кук", "ячм", "овес", "пшеница", "кукурузное зерно",
        "мука кукурузная", "мука ячменная", "кукуруза молотая", "дерть", "плющенка", "кк энергетический",
        "энергетик", "карб", "крахмал", "солод", "зерносмесь", "мука пшеничная", "тритикале", "тритикале плющеная"
    ],
    "protein": [
        "шрот", "жмых", "соев", "соевый", "протеин", "оболочка сои", "соя", "люпин", "горох", "бобы",
        "белок", "лен", "дрожи",
        "шр", "соево", "соевый шрот", "rapeseed meal", "sunflower meal", "подсолн", "подсолнечный шрот",
        "шр подсолнечный", "экспандат", "экспеллер", "шрот соевый", "соевый жмых",
        "жмых рапсовый", "жмых льняной", "мучка", "мука протеиновая", "соя экструдированная",
        "bean", "peas", "protein meal", "чечевица"
    ],
    "fat": [
        "жир", "масло", "пропионат", "жир защищённый", "масло соевое",
        "триглицериды", "жир кормовой", "жир растительный", "жир животный", "жир инертный",
        "жир корм", "пальмовое масло", "РЖК", "protected fat", "жирный концентрат"
    ],
    "premix": [
        "премикс", "премикс дойн", "премикс молод", "премикс транзит",
        "premix", "пре", "мин-премикс", "вит-премикс", "премикс для коров",
        "премикс для телят", "премикс уни", "пр.дойный"
    ],
    "mineral_vit": [
        "мел", "соль", "витамин", "минерал", "извест", "кальций", "фосфат",
        "лизунец", "минвит", "сода", "поташ", "галит брикеты",
        "Ca", "P", "NaCl", "CaCO3", "минсмесь", "витамины", "минералы",
        "ди кальций фосфат", "морская соль", "лизун", "лизунцы", "соль кормовая",
        "сода корм", "кальцит", "лед",  "soda"
    ],
    "additive": [
        "дрожж", "пробиот", "фермент", "энзим", "энзимы", "буфер", "аквасейф", "биотек",
        "мегабуст", "электролит", "кисол", "шаумацид дринк",
        "пробиотик", "бактерии", "закваска", "пребиотик", "аминокислота",
        "сорбент", "адсорбент", "консервант", "антиоксидант", "детокс", "детоксикант",
        "биостимулятор", "acid", "enzyme", "холин", "провилит", "ампролиум"
    ],
    "byproduct": [
        "жом", "барда", "жом свекловичный", "сыворотка", "дробина", "зeрноотход",
        "остатк", "остатки",
        "пивная дробина", "барда сухая", "жом сухой", "жом гранулированный",
        "DDGS", "ддгс", "зерноотходы", "побочка", "отруби", "брикет жома",
        "меласса", "oстатки", "сухая стружка"
    ],
    "water": [
        "вода", "влажн", "сыворотка молочная", "молоко",
        "H2O", "жидкость", "влажное сырье", "сыворотк", "сыв", "обрат"
    ],
    "premix_blend": [
        "комбикорм", "кк", "кк№", "кк №", "комбикорм №",
        "стартер", "престартер", "бустер милк", "кормосмесь №10 снт",
        "кальвобустер", "бвмк для коров", "зцм",
        "ККР", "концентрат", "смесь", "БВМК", "ПК", "концентрат белковый",
        "концентрат энергетический", "комби", "смесь кормовая",
        "complete feed", "feed mix", "starter feed", "grower", "зск", "бвмк", "kkd1"
    ]
}


In [93]:
def classify_ingredient(name):
    if not isinstance(name, str):
        return "other"
    name_lower = name.lower()
    for group, kat_list in GROUP_KEYWORDS.items():
        for kat in kat_list:
            if kat in name_lower:
                return group
    return "other"
feeding_details_all['ingredient_group'] = feeding_details_all['decoded_name'].apply(classify_ingredient)
feeding_details_all[['norm_ingr_name', 'decoded_name', 'ingredient_group']].head(20)

,norm_ingr_name,decoded_name,ingredient_group
0,солома покупная,солома покупная,forage
1,3645.01.01.01.1.20,люцерна сенаж,forage
2,комбикорм 10 группы,комбикорм 10 группы,premix_blend
3,кукуруза сухая,кукуруза сухая,energy
4,шрот подсолнечный,шрот подсолнечный,protein
5,шрот соевый,шрот соевый,protein
6,жмых рапсовый,жмых рапсовый,protein
7,жир защищённый,жир защищённый,fat
8,2262.01.05.02.1.21,кукуруза силос,forage
9,вода,вода,water


In [94]:
print(feeding_details_all.ingredient_group.value_counts())

ingredient_group
forage          5963982
energy          2873289
protein         2664047
water            971809
byproduct        962749
premix_blend     943903
premix           935655
mineral_vit      672546
fat              264034
additive         191061
other                 4
Name: count, dtype: int64


In [95]:
others = feeding_details_all[feeding_details_all['ingredient_group'] == 'other']
others['decoded_name'].value_counts()

decoded_name
    4
Name: count, dtype: int64

In [ ]:
others[others["decoded_name"] == ""]

,FeedingTaskID,SectionID,PhysiologicalGroupID,PhysiologicalGroupName,IngredientID,IngredientName,IngredientType,PhysicalWeight_kg,year,norm_ingr_name,is_code,decoded_name,ingredient_group
11574693,Farms/EkoNiva1C.9d4645a7-570c-11e2-9cc0-00155d...,342,54,Т1 (3-5 мес.),260,NaN,Premixture,0.0,2024,,False,,other
11592293,Farms/EkoNiva1C.9d4645a7-570c-11e2-9cc0-00155d...,342,54,Т1 (3-5 мес.),260,NaN,Premixture,0.0,2024,,False,,other
11608364,Farms/EkoNiva1C.9d4645a7-570c-11e2-9cc0-00155d...,342,54,Т1 (3-5 мес.),260,NaN,Premixture,0.0,2024,,False,,other
11622461,Farms/EkoNiva1C.9d4645a7-570c-11e2-9cc0-00155d...,342,54,Т1 (3-5 мес.),260,NaN,Premixture,0.0,2024,,False,,other


In [109]:
feeding_details_all

,FeedingTaskID,SectionID,PhysiologicalGroupID,PhysiologicalGroupName,IngredientID,IngredientName,IngredientType,PhysicalWeight_kg,year,norm_ingr_name,is_code,decoded_name,ingredient_group
0,Farms/EkoNiva1C.216d4235-2852-11e8-80c4-1c98ec...,9,2,Д1,125,Солома покупная,Forage,150.259,2022,солома покупная,False,солома покупная,forage
1,Farms/EkoNiva1C.216d4235-2852-11e8-80c4-1c98ec...,9,2,Д1,773,3645.01.01.01.1.20,Forage,880.444,2022,3645.01.01.01.1.20,True,люцерна сенаж,forage
2,Farms/EkoNiva1C.216d4235-2852-11e8-80c4-1c98ec...,9,2,Д1,72,Комбикорм 10 группы,Concentrate,542.257,2022,комбикорм 10 группы,False,комбикорм 10 группы,premix_blend
3,Farms/EkoNiva1C.216d4235-2852-11e8-80c4-1c98ec...,9,2,Д1,82,Кукуруза сухая,Concentrate,991.793,2022,кукуруза сухая,False,кукуруза сухая,energy
4,Farms/EkoNiva1C.216d4235-2852-11e8-80c4-1c98ec...,9,2,Д1,129,Шрот подсолнечный,Concentrate,156.054,2022,шрот подсолнечный,False,шрот подсолнечный,protein
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16443074,Farms/EkoNiva1C.6c69fe1b-2b6f-11e8-80c4-1c98ec...,131,1,Нетели,119,Солома (общ.),Forage,120.643,2025,солома (общ.),False,солома (общ.),forage
16443075,Farms/EkoNiva1C.6c69fe1b-2b6f-11e8-80c4-1c98ec...,131,1,Нетели,335,6203.01.01.01.1.25,Forage,549.169,2025,6203.01.01.01.1.25,True,люцерна сенаж,forage
16443076,Farms/EkoNiva1C.6c69fe1b-2b6f-11e8-80c4-1c98ec...,131,1,Нетели,331,Жом свекловичный сухой,Concentrate,36.676,2025,жом свекловичный сухой,False,жом свекловичный сухой,byproduct
16443077,Farms/EkoNiva1C.6c69fe1b-2b6f-11e8-80c4-1c98ec...,131,1,Нетели,143,Премикс молодняк 6-24,VitaminMineral,10.049,2025,премикс молодняк 6-24,False,премикс молодняк 6-24,premix


In [112]:
feeding_details_all = feeding_details_all[feeding_details_all["ingredient_group"] != "other"]

In [113]:
feeding_details_all.to_csv("out.csv")